In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

2023-09-23 17:19:08.780228: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-23 17:19:08.782335: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 17:19:08.813140: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-23 17:19:08.813689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 17:19:09.581207: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
data = pd.read_csv("data.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
0,0,2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.396
1,1,2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.385
2,2,2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.379
3,3,2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.368
4,4,2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.375


In [3]:
data.drop("Unnamed: 0.1", axis = 1, inplace = True)
data.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
data["Date"] = pd.to_datetime(data["Date"])
data.set_index("Date", inplace=True)
data.replace(0, np.nan, inplace=True)

In [4]:
data

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.3960
2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.3850
2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.3790
2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.3680
2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.6978,1.1174,4.0507,1.3073,6.9954,6.6829,7.7874,71.45,109.47,4.1260
2019-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.interpolate(method='linear', limit_direction='forward')

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.913300,1.441900,1.720000,1.0377,6.8273,5.1597,7.755500,46.27,92.550000,3.3960
2010-01-05,0.914300,1.440200,1.729600,1.0371,6.8258,5.1668,7.756400,46.13,91.480000,3.3850
2010-01-06,0.918900,1.440400,1.729200,1.0333,6.8272,5.1638,7.754600,45.72,92.530000,3.3790
2010-01-07,0.916800,1.431400,1.740900,1.0351,6.8280,5.1981,7.753900,45.67,93.310000,3.3680
2010-01-08,0.921800,1.435700,1.734200,1.0345,6.8274,5.1827,7.755300,45.50,92.700000,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.697800,1.117400,4.050700,1.3073,6.9954,6.6829,7.787400,71.45,109.470000,4.1260
2019-12-28,0.698667,1.118833,4.038867,1.3068,6.9924,6.6749,7.786833,71.40,109.263333,4.1191
2019-12-29,0.699533,1.120267,4.027033,1.3063,6.9894,6.6669,7.786267,71.35,109.056667,4.1122


In [6]:
data.interpolate(method='linear', limit_direction='forward', inplace=True)
LOOK_BACK = 30
PREDICT_DAY = 1
SPLIT_RATIO = 0.8

In [7]:
def Create_Data(data, lookback = LOOK_BACK, pred_len = PREDICT_DAY, split_ratio = SPLIT_RATIO):
    if lookback<2:
        print("ERROR: Lookback too small")
        return -1

# declarations

    x = {}
    y = {}
    xtr = {}
    xt = {}
    ytr = {}
    yt = {}
    scalers = {}

# Creating stepped data

    for i in data.columns:
        xtemp = pd.DataFrame(data[i])
        for j in range(1,lookback+1):
            xtemp[i+str(j)] = data[i].shift(-1*j)
        x[i] = xtemp.dropna()

# Splitting data into x and y
        
    for i in x.keys():
        y[i] = pd.DataFrame(x[i].iloc[:,-pred_len])
        x[i] = x[i].iloc[:,:-pred_len]
        
# Normalizing x and y values
        
    for i in x.keys():
        scalers[i+"_x"] = MinMaxScaler(feature_range=(0,1))
        x[i] = scalers[i+"_x"].fit_transform(x[i])
        scalers[i+"_y"] = MinMaxScaler(feature_range=(0,1))
        y[i] = scalers[i+"_y"].fit_transform(y[i])
        
# setting train and test sizes
        
    tr_len = int(split_ratio*y["India"].shape[0])
    t_len = y["India"].shape[0]-tr_len
    
# creating training and testing data
    
    for i in x.keys():
        xtr[i] = x[i][:tr_len]
        ytr[i] = y[i][:tr_len]
        xt[i] = x[i][-t_len:]
        yt[i] = y[i][-t_len:]
        
# returning pertinent data
    
    return x,y,xtr,xt,ytr,yt,scalers

In [8]:
x,y,xtr,xt,ytr,yt,scalers = Create_Data(data)

In [9]:
def Create_model(x,y, lookback = LOOK_BACK, Pred_size = PREDICT_DAY):
    models = {}
    for i in x.keys():
        models[i] = Sequential()
        models[i].add(Dense(32,input_shape=(LOOK_BACK,),activation="relu"))
        models[i].add(Dense(64,activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(Dense(128,activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(Dense(64,activation="relu"))
        models[i].add(Dropout(0.2))
        models[i].add(Dense(16,activation="relu"))
        models[i].add(Dense(Pred_size))
        models[i].compile(loss="mean_squared_error", optimizer="adam")
        print(i)
    return models

In [10]:
m = Create_model(x,y)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                992       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

None

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                992       
                                                                 
 dense_7 (Dense)             (None, 64)                2112      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 128)               8320      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)               

None

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 32)                992       
                                                                 
 dense_13 (Dense)            (None, 64)                2112      
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 128)               8320      
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dropout_8 (Dropout)         (None, 64)               

...

None

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 32)                992       
                                                                 
 dense_49 (Dense)            (None, 64)                2112      
                                                                 
 dropout_24 (Dropout)        (None, 64)                0         
                                                                 
 dense_50 (Dense)            (None, 128)               8320      
                                                                 
 dropout_25 (Dropout)        (None, 128)               0         
                                                                 
 dense_51 (Dense)            (None, 64)                8256      
                                                                 
 dropout_26 (Dropout)        (None, 64)               

None

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 32)                992       
                                                                 
 dense_55 (Dense)            (None, 64)                2112      
                                                                 
 dropout_27 (Dropout)        (None, 64)                0         
                                                                 
 dense_56 (Dense)            (None, 128)               8320      
                                                                 
 dropout_28 (Dropout)        (None, 128)               0         
                                                                 
 dense_57 (Dense)            (None, 64)                8256      
                                                                 
 dropout_29 (Dropout)        (None, 64)               

None

In [28]:
def Execute_model(model,xtr,ytr,xt,yt, scaler):
    MAPE = {}
    MAE = {}
    MSE = {}
    for i in model.keys():
        print(i)
# Training the model
        
        model[i].fit(xtr[i], ytr[i], epochs=10, batch_size=1, verbose=1)
        
# collecting predicted and actual values
        
        temp = model[i].predict(xt[i])
        pred = scaler[i+"_y"].inverse_transform(temp)
        act = scaler[i+"_y"].inverse_transform(yt[i])
        
# calculating Mean Square Error, Mean Absolute Error, and Mean Absolute Error

        MSE[i] = mean_squared_error(act,pred)
        MAE[i] = mean_absolute_error(act,pred)
        MAPE[i] = mean_absolute_percentage_error(act, pred)
        
# Tabulating Data

    results = pd.DataFrame([MSE,MAE,MAPE])
    results["Metric"] = ["MSE","MAE","MAPE"]
    results.set_index("Metric",inplace=True)
    
    return results


In [27]:
result = Execute_model(m,xtr,ytr,xt,yt,scalers)

Epoch 1/10
2894/2894 [==============================] - 2s 767us/step - loss: 0.0015
Epoch 2/10
2894/2894 [==============================] - 2s 749us/step - loss: 0.0014
Epoch 3/10
2894/2894 [==============================] - 2s 747us/step - loss: 0.0014
Epoch 4/10
2894/2894 [==============================] - 2s 748us/step - loss: 0.0013
Epoch 5/10
2894/2894 [==============================] - 2s 786us/step - loss: 0.0013
Epoch 6/10
2894/2894 [==============================] - 2s 746us/step - loss: 0.0013
Epoch 7/10
2894/2894 [==============================] - 2s 761us/step - loss: 0.0011
Epoch 8/10
2894/2894 [==============================] - 2s 748us/step - loss: 0.0012
Epoch 9/10
2894/2894 [==============================] - 2s 747us/step - loss: 0.0011
Epoch 10/10
23/23 [==============================] - 0s 623us/step
Epoch 1/10
2894/2894 [==============================] - 2s 748us/step - loss: 0.0016
Epoch 2/10
2894/2894 [==============================] - 2s 800us/step - loss: 0.001

2894/2894 [==============================] - 4s 1ms/step - loss: 0.0014
Epoch 3/10
2894/2894 [==============================] - 4s 1ms/step - loss: 0.0013
Epoch 4/10
2894/2894 [==============================] - 4s 1ms/step - loss: 0.0013
Epoch 5/10
2894/2894 [==============================] - 3s 1ms/step - loss: 0.0014
Epoch 6/10
2894/2894 [==============================] - 3s 1ms/step - loss: 0.0011
Epoch 7/10
2894/2894 [==============================] - 4s 1ms/step - loss: 0.0011
Epoch 8/10
2894/2894 [==============================] - 4s 1ms/step - loss: 0.0011
Epoch 9/10
2894/2894 [==============================] - 4s 1ms/step - loss: 0.0012
Epoch 10/10
23/23 [==============================] - 0s 878us/step


In [29]:
results

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Metric,,,,,,,,,,
MSE,0.000080,0.000085,0.014046,0.000249,0.009263,0.001546,0.000027,9.054012,0.439475,0.001273
MAE,0.007873,0.007414,0.099795,0.014213,0.079726,0.031106,0.004025,2.831270,0.493226,0.024860
MAPE,0.011022,0.006412,0.025454,0.010778,0.011545,0.004840,0.000514,0.040358,0.004520,0.006225
